
## [Reference Notebook by NotebookUtils](https://learn.microsoft.com/en-us/fabric/data-engineering/notebook-utilities#reference-a-notebook)

This method references a notebook and returns its exit value. You can run nesting function calls in a notebook interactively or in a pipeline. The notebook being referenced runs on the Spark pool of the notebook that calls this function.

### reference a single notebook

```notebookutils.notebook.run("notebook name", <timeoutSeconds>, <parameterMap>, <workspaceId>)```


In [26]:
 %run childNotebook1 { 'parameter1': 'value1', 'parameter2': 'value2'}

StatementMeta(, 9e70e3d5-7cab-4b3d-b694-b1eb555eb447, 31, Finished, , Finished)

this is child notebook with parameter1 = value1	parameter2 = value2
ExitValue: exit with current Notebook Name: MainNotebook

In [27]:
exitVaule = notebookutils.notebook.run('childNotebook1', 100, {'parameter1': 'value1', 'parameter2': 'value2'})
print(f'exit vaule = {exitVaule}')

StatementMeta(, 9e70e3d5-7cab-4b3d-b694-b1eb555eb447, 32, Finished, Available, Finished)

exit vaule = exit with current Notebook Name: childNotebook1


### Reference run multiple notebooks in parallel

The method ```notebookutils.notebook.runMultiple()``` allows you to run multiple notebooks in parallel or with a predefined topological structure. The API is using a multi-thread implementation mechanism within a spark session, which means the reference notebook runs share the compute resources. code example:

```
notebookutils.notebook.runMultiple(["NotebookSimple", "NotebookSimple2"])
```

In [15]:
import json

exitCodes = notebookutils.notebook.runMultiple(["childNotebook1", "childNotebook2"])
print(json.dumps(exitValues, indent=4))

StatementMeta(, 9e70e3d5-7cab-4b3d-b694-b1eb555eb447, 17, Finished, Available, Finished)

{
    "activity1": {
        "exitVal": "my exit valule: value1\tvalue1",
        "exception": null
    },
    "activity2": {
        "exitVal": "exit with current Notebook Name: childNotebook2",
        "exception": null
    }
}


In [16]:
# run multiple notebooks with parameters
import json
dag = {
    'activities': [
        {
            'name': 'activity1', # activity name, must be unique
            'path': 'childNotebook1', # notebook path
            'timeoutPerCellInSeconds': 90, # max timeout for each cell, default to 90 seconds
            'args': {'parameter1': 'value1', 'parameter2': 'value1'}, # notebook parameters
        },
        {
            'name': 'activity2',
            'path': 'childNotebook2',
            'timeoutPerCellInSeconds': 120,
            'args': {'input1': 'changed value 2', 'input2': 200}
        }
    ],
    'timeoutInSeconds': 43200, # max timeout for the entire DAG, default to 12 hours
    'concurrency': 50 # max number of notebooks to run concurrently, default to 50
}

notebookutils.notebook.validateDAG(dag)

exitValues = notebookutils.notebook.runMultiple(dag, {'displayDAGViaGraphviz': False})

print(json.dumps(exitValues, indent=4))

StatementMeta(, 9e70e3d5-7cab-4b3d-b694-b1eb555eb447, 18, Finished, Available, Finished)

{
    "activity1": {
        "exitVal": "my exit valule: value1\tvalue1",
        "exception": null
    },
    "activity2": {
        "exitVal": "exit with current Notebook Name: childNotebook2",
        "exception": null
    }
}


In [21]:
from multiprocessing.pool import ThreadPool

def reference_notebook(name: str) -> str:
    return notebookutils.notebook.run(name)

names = ['childNotebook1', 'childNotebook2']
# Create a ThreadPool with 3 worker threads

with ThreadPool(3) as pool:
    results = pool.map(reference_notebook, names)

# Print the results
print("reference run results: ", results)

StatementMeta(, 9e70e3d5-7cab-4b3d-b694-b1eb555eb447, 23, Finished, Available, Finished)

reference run results:  ['my exit valule:', 'exit with current Notebook Name: childNotebook2']


### [Notebook resources](https://learn.microsoft.com/en-us/fabric/data-engineering/how-to-use-notebook#notebook-resources)
The notebook resource explorer provides a Unix-like file system to help you manage your folders and files. It offers a writeable file system space where you can store small-sized files, such as code modules, semantic models, and images. You can easily access them with code in the notebook as if you were working with your local file system.

In [25]:
import builtin.my as my
# Now use the exported members from this module with identifier: `my`
my.func()

StatementMeta(, 9e70e3d5-7cab-4b3d-b694-b1eb555eb447, 27, Finished, Available, Finished)

this is a function in my own module
